# DataFrames para funciones y sus respectivas funciones.

En esta sección [Data_Funciones], se realizarán las reducciones pertinentes a cada DataFrame para luego guardarlos en archivos parquet y utilizarlos más tarde con nuestra API. Además, se implementarán las funciones a emplear en Fast API.

### Importamos librerías

Estas librerías nos permiten manipular los datos y almacenarlos.

In [3]:
import pandas as pd
import sys
sys.path.insert(0, '../')
import Herramientas as Herr

### Carga de Datos

In [4]:
data_review = pd.read_csv('../datasets/australian_reviews_listo.csv')
data_items = pd.read_csv('../datasets/australian_items.csv')
data_output= pd.read_csv('../datasets/output.csv')

### PlayTimeGenre

#### Procesamiento de datos

Se unió el DataFrame de los juegos (data_output) con el de items que posee las horas jugadas de cada juego. Luego se eliminaron los datos nulos que se generaron en la fusión y aquellas filas que contenían al menos un valor nulo. Esta decisión se tomó para asegurar que los datos restantes fueran confiables. Además, eliminamos otras columnas que no son relevantes para la función.

In [5]:
data_genres = pd.merge(data_output,data_items,on='item_id',how='right')
data_genres = data_genres.drop(columns=['publisher','app_name','title','price','early_access','developer','item_name','user_id','items_count','steam_id'])
data_genres = data_genres.dropna()

Seleccionamos aquellos juegos y sus géneros que tenían más de 0 horas jugadas.

In [6]:
data_genres = data_genres.loc[data_genres['playtime_forever'] > 0]
data_genres

,genres,release_date,item_id,playtime_forever
0,Action,2000,10,6.0
2,Action,2003,30,7.0
8,Action,2010,300,4733.0
9,Action,2004,240,1853.0
10,Action,2005,3830,333.0
...,...,...,...,...
10065958,Indie,2016,227940,43.0
10065959,Massively Multiplayer,2016,227940,43.0
10065964,Adventure,2015,388490,3.0
10065965,Casual,2016,521570,4.0


Se agruparon los registros por género y fecha de lanzamiento, además se sumaron la cantidad de horas jugadas. Luego se reseteó el índice para que los identificadores estén con un orden lógico.

In [7]:
data_playtime = data_genres.groupby(['genres','release_date'])['playtime_forever'].sum().reset_index()
data_playtime

,genres,release_date,playtime_forever
0,Action,1983,3582.0
1,Action,1984,384.0
2,Action,1988,16243.0
3,Action,1989,607.0
4,Action,1990,18787.0
...,...,...,...
322,Web Publishing,2013,335849.0
323,Web Publishing,2014,33732.0
324,Web Publishing,2015,348861.0
325,Web Publishing,2016,136.0


### Función

La función "PlayTimeGenre" tiene dos parámetros que son: un DataFrame y un género que debe ser de tipo string. Se filtra el DataFrame a través del género, se ordenan los valores de forma descendente y se obtiene solo el primer valor, lo mismo para el año de posteo.

In [8]:
def PlayTimeGenre (dataframe,genero:str):
    data_play = dataframe[dataframe['genres'] == genero]
    data_play = data_play.sort_values(by='playtime_forever',ascending=False).head(1)
    
    date = data_play['release_date'].values[0]
    feedback = (f'Año de lanzamiento con más horas jugadas para Género {genero}: {date}')
    return feedback

In [9]:
PlayTimeGenre(data_genres,'Indie')

'Año de lanzamiento con más horas jugadas para Género Indie: 2006'

### UsersWorstDeveloper

#### Procesamiento de datos

Se verificaron dos igualdades: si la recomendación era falsa y si el comentario era negativo. Se agruparon los registros de esas igualdades a través de su identificador (item_id) y el año de posteo, además de resetear el índice. Luego se unió el DataFrame "count_worst_reviews" con el DataFrame "data_output". Se quitaron columnas innecesarias para la función.

In [10]:
worst_reviews = data_review[(data_review['recommend'] == False) & (data_review['sentiment_analysis'] == 0)]
count_worst_reviews = worst_reviews.groupby(['item_id','posted']).size().reset_index(name='cbad_reviews')
df_users = pd.merge(count_worst_reviews,data_output)
df_users = df_users.drop(columns=['early_access','price','release_date','app_name','title','genres','publisher'])
df_users


,item_id,posted,cbad_reviews,developer
0,10,2015,1,Valve
1,20,2015,1,Valve
2,80,2014,1,Valve
3,220,2014,1,Valve
4,220,2015,1,Valve
...,...,...,...,...
1539,501760,2014,1,Kiddy
1540,501760,2014,1,Kiddy
1541,501760,2014,1,Kiddy
1542,502550,2014,1,Freakinware Limited


#### Función

La función "UsersWorstDeveloper" recibe dos parámetros: un DataFrame y el año que debe ser de tipo entero. Se instancia un nuevo DataFrame filtrado con solo el año que se pide. Luego, se seleccionan solo los 3 primeros desarrolladores que nos da el contador de valores, y se hace un ciclo for para devolver en qué posición y qué desarrolladores son los 3 peores.

In [11]:
def UsersWorstDeveloper(dataframe,anio:int):

    dfworst = dataframe[(dataframe['posted'] == anio)]

    count_worst_dev = dfworst['developer'].value_counts()

    top_3_worst = count_worst_dev.head(3)

    top_3 = []

    for position,(developer,_) in enumerate(top_3_worst.items()):
        top_3.append({f'Puesto {position+1}:':developer})

    return top_3

In [12]:
UsersWorstDeveloper(df_users,2015)

[{'Puesto 1:': 'Cherry Pop Games'},
 {'Puesto 2:': 'ProjectorGames'},
 {'Puesto 3:': 'The Behemoth'}]

### UserForGenre

#### Procesamiento de datos

In [13]:
data_user_genre = pd.merge(data_output,data_items)
data_user_genre = data_user_genre.drop(columns=['publisher','app_name','title','price','early_access','developer','item_name','items_count','steam_id'])
data_user_genre

,genres,release_date,item_id,playtime_forever,user_id
0,Action,1997,282010,5.0,UTNerd24
1,Action,1997,282010,0.0,I_DID_911_JUST_SAYING
2,Action,1997,282010,0.0,76561197962104795
3,Action,1997,282010,0.0,r3ap3r78
4,Action,1997,282010,13.0,saint556
...,...,...,...,...,...
9115519,Action,2004,80,0.0,76561198273508956
9115520,Action,2004,80,0.0,76561198282090798
9115521,Action,2004,80,0.0,943525
9115522,Action,2004,80,9.0,76561198283312749


In [14]:
data_user_genre = data_user_genre.groupby(['user_id','genres','release_date'])['playtime_forever'].sum().reset_index()
data_user_genre

,user_id,genres,release_date,playtime_forever
0,--000--,Action,2009,5329.0
1,--000--,Action,2010,22.0
2,--000--,Action,2011,6522.0
3,--000--,Action,2012,109346.0
4,--000--,Action,2013,363.0
...,...,...,...,...
3103381,zzzmidmiss,Strategy,2009,0.0
3103382,zzzmidmiss,Strategy,2010,10.0
3103383,zzzmidmiss,Strategy,2011,69.0
3103384,zzzmidmiss,Strategy,2012,923.0


In [15]:
data_user_genre = data_user_genre.loc[data_user_genre['playtime_forever']>0]
data_user_genre

,user_id,genres,release_date,playtime_forever
0,--000--,Action,2009,5329.0
1,--000--,Action,2010,22.0
2,--000--,Action,2011,6522.0
3,--000--,Action,2012,109346.0
4,--000--,Action,2013,363.0
...,...,...,...,...
3103379,zzzmidmiss,Sports,2010,194.0
3103380,zzzmidmiss,Sports,2014,16.0
3103382,zzzmidmiss,Strategy,2010,10.0
3103383,zzzmidmiss,Strategy,2011,69.0


def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

### Función

In [28]:
def UserForGenre(data_user_genre,genero:str):

    data_genre = data_user_genre[data_user_genre['genres'] == genero]
    data_usuario = data_genre.sort_values(by='playtime_forever',ascending=False).iloc[0]
    hours = data_usuario.values[0]
    data_user =data_usuario.values[0]
    anio = data_usuario.values[0]
    horas_anuales_usuario = data_genre.groupby('release_date')['playtime_forever'].sum().reset_index()
    horas_anuales_ = horas_anuales_usuario.to_dict(orient='records')
    write = (f'Usuario con más horas jugadas para el género {genero} es: {data_user}, en el año {anio}: {hours} jugadas, además tenemos las horas jugadas por año del usuario {data_user}: {horas_anuales_}')
    return write              

In [33]:
UserForGenre(data_user_genre,'Sports')

"Usuario con más horas jugadas para el género Sports es: 76561198019112245, en el año 76561198019112245: 76561198019112245 jugadas, además tenemos las horas jugadas por año del usuario 76561198019112245: [{'release_date': '1995', 'playtime_forever': 31220.0}, {'release_date': '2005', 'playtime_forever': 69758.0}, {'release_date': '2006', 'playtime_forever': 11211.0}, {'release_date': '2007', 'playtime_forever': 1308.0}, {'release_date': '2008', 'playtime_forever': 98946.0}, {'release_date': '2009', 'playtime_forever': 250785.0}, {'release_date': '2010', 'playtime_forever': 2397026.0}, {'release_date': '2011', 'playtime_forever': 930765.0}, {'release_date': '2012', 'playtime_forever': 3608452.0}, {'release_date': '2013', 'playtime_forever': 6257701.0}, {'release_date': '2014', 'playtime_forever': 5604627.0}, {'release_date': '2015', 'playtime_forever': 41795973.0}, {'release_date': '2016', 'playtime_forever': 4763913.0}, {'release_date': '2017', 'playtime_forever': 22876.0}, {'release_d